In [22]:
import pandas as pd
import re
import demoji

In [23]:
from flask import Flask, jsonify

from flask import request
from flasgger import Swagger, LazyString, LazyJSONEncoder
from flasgger import swag_from

In [25]:
app = Flask(__name__)

app.json_encoder = LazyJSONEncoder

swagger_template = {
    "info": {
        "title": "API and SwaggerUI for Gold Challenge",
        "version": "1.0.0",
        "description": "API dan SwaggerUI untuk Gold Challenge",
    },
    "host": "127.0.0.1:5000"
}

swagger_config = {
    "headers": [],
    "specs": [
        {
            "endpoint": 'docs',
            "route": '/docs.json',
        }
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui": True,
    "specs_route": "/docs/"
}


swagger = Swagger(app, template=swagger_template,             
                  config=swagger_config)

@swag_from("/Users/aquar/Binar Academy/Gold Challenge/text_processing.yml", methods=['POST'])
@app.route('/text-processing', methods=['POST'])
def text_processing():

    text = request.form.get('text')
    
    def remove_unicode_series(sentence):
        result_no_x = re.sub(r'\\x..', ' ', sentence)
        result_no_unicode = re.sub(r'\ufffd', ' ', result_no_x)
        result_no_user = re.sub(r'USER', ' ', result_no_unicode)
        result_no_backslash_n = re.sub(r'\\n', ' ', result_no_user)
        result_no_symbol = re.sub(r'[^a-zA-Z0-9]', ' ', result_no_backslash_n)
        return result_no_symbol
    
    text2 = remove_unicode_series(text)
    
    abusivelist = pd.read_csv('abusive.csv')['ABUSIVE'].to_list()
    
    abusive_list = []   
    
    for i in abusivelist:
        i = ' ' * 1 + i + ' ' * 1
        abusive_list.append(i)
        
    def remove_abusive(sentence, abusive):
        sentence = sentence.lower()
        for i in abusive:
            sentence = re.sub(i, ' ', sentence)
        return sentence
    
    text3 = remove_abusive(text2, abusive = abusive_list)
    
    new_kamusalay = pd.read_csv('new_kamusalay.csv', names = ['Kata_Alay', 'Kata_Formal'], encoding = 'windows-1250')
    
    kamusalay = new_kamusalay['Kata_Alay'].to_list()
    
    kamus_alay = []

    for i in kamusalay:
        i = ' ' * 1 + i + ' ' * 1
        kamus_alay.append(i)

    kamus_alay
    
    kamusformal = new_kamusalay['Kata_Formal'].to_list()
    
    kamus_formal = []

    for j in kamusformal:
        j = ' ' * 1 + j + ' ' * 1
        kamus_formal.append(j)

    kamus_formal
    
    def change_alay_to_formal(sentence, alay, formal):
        for i, j in zip(alay, formal):
            sentence = re.sub(i, j, sentence)
        return sentence
    
    text4 = change_alay_to_formal(text3, alay = kamus_alay, formal = kamus_formal)
    
    json_response = {
        'status_code': 200,
        'description': "Teks yang sudah diproses",
        'data': text4,
    }
    
@swag_from("/Users/aquar/Binar Academy/Gold Challenge/text_processing_file.yml", methods=['POST'])
@app.route('/text-processing-file', methods=['POST'])

def file_processing():
    
    file = request.files.getlist('file')[0]
    
    data = pd.read_csv(file, encoding = 'windows-1250')
    
    abusivelist = pd.read_csv('abusive.csv')['ABUSIVE'].to_list()
    
    abusive_list = []   
    
    for i in abusivelist:
        i = ' ' * 1 + i + ' ' * 1
        abusive_list.append(i)
    
    def remove_unicode_series(sentence):
        result_no_x = re.sub(r'\\x..', '', sentence)
        result_no_unicode = re.sub(r'\ufffd', '', result_no_x)
        result_no_user = re.sub(r'USER', '', result_no_unicode)
        result_no_backslash_n = re.sub(r'\\n', '', result_no_user)
        return result_no_backslash_n
    
    data['Tweet'] = data['Tweet'].apply(remove_unicode_series)
    
    data['Tweet'] = data['Tweet'].apply(remove_abusive, abusive = abusive_list)
    
    new_kamusalay = pd.read_csv('new_kamusalay.csv', names = ['Kata_Alay', 'Kata_Formal'], encoding = 'windows-1250')
    
    kamusalay = new_kamusalay['Kata_Alay'].to_list()
    
    kamus_alay = []

    for i in kamusalay:
        i = ' ' * 1 + i + ' ' * 1
        kamus_alay.append(i)
        
    kamusformal = new_kamusalay['Kata_Formal'].to_list()
    
    kamus_formal = []

    for j in kamusformal:
        j = ' ' * 1 + j + ' ' * 1
        kamus_formal.append(j)
        
    def change_alay_to_formal(sentence, alay, formal):
        for i, j in zip(alay, formal):
            sentence = re.sub(i, j, sentence)
        return sentence
    
    data['Tweet'] = data['Tweet'].apply(change_alay_to_formal, alay = kamus_alay, formal = kamus_formal)
    
    json_response = {
        'status_code': 200,
        'description': "Teks yang sudah diproses",
        'data': data,
    }

    response_data = jsonify(json_response)
    return response_data

if __name__ == '__main__':
   app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
